# UQpy at FrontUQ 2024
## Polynomial Chaos Expansion

This notebook trains a [Polynomial Chaos Expansion](https://en.wikipedia.org/wiki/Polynomial_chaos) (PCE) as a surrogate model for the aerodynamic model defined by XFoil. The outline of this notebook is below.

1. Compute the training data using UM-Bridge calls to the XFoil model
2. Define and train a PCE for all four outputs
3. Compare PCE and Monte Carlo statistics and Sobol sensitivities
4. Plot the results

First, we import the necessary packages. We recommend you display the UQpy logs, but it is optional and does not affect the functionality of any code.

In [ ]:
import numpy as np
import UQpy as uq
import umbridge

np.random.seed(0)

import logging  # Optional, display UQpy logs

logger = logging.getLogger("UQpy")
logger.setLevel(logging.INFO)

## 1. Generate training and testing data
We define the inputs using random samples generated by UQpy and compute the outputs using UM-Bridge to call the XFoil model. Note that you can generate random inputs using the `.rvs` method in UQpy or use a more sophisticated [sampling scheme](https://uqpyproject.readthedocs.io/en/latest/sampling/index.html). Latin hypercube sampling is shown below.

Once all `n` samples are generated, we treat the first half as training data the surrogate will train on and the second half as a dataset to test the performance of the surrogate.

In [ ]:
# define inputs using UQpy
marginals = [
    uq.Normal(0.0, 0.1),
    uq.Normal(500_000, 2_500),
    uq.Normal(0.3, 0.015),
    uq.Normal(0.7, 0.021),
    uq.Normal(0, 0.08),
]
distribution = uq.JointIndependent(marginals)
n = 100
x = distribution.rvs(n)  # random samples
# x = uq.LatinHypercubeSampling(distribution, n, random_state=0).samples  # LHS samples

# compute outputs using UM-Bridge
model = umbridge.HTTPModel("http://localhost:53687", "forward")
outputs = np.zeros((n, 4))
for i in range(n):
    if i % (n // 10) == 9:
        print(f"UM-Bridge Model Evaluations: {i + 1} / {n}")
    model_input = [x[i].tolist()]  # model input is a list of lists
    outputs[i] = model(model_input)[0]

# split into training and testing data sets
split = int(n * 0.5)
x_train = x[0:split]
y_train = outputs[0:split]
x_test = x[split:]
y_test = outputs[split:]

## 2. Train Polynomial Chaos Expansion

We build a PCE model for all four outputs. PCEs build a polynomial approximation of the system. We use a low order polynomial since the XFoil model is roughly linear. It is possible to include higher order terms in the expansion, but quickly because prohibitively expensive to train. One attempt to mitigate the cost of higher order terms is through the use of [sparse polynomial basis](https://www.sciencedirect.com/science/article/pii/S0021999110006856?via%3Dihub), which are [implemented in UQpy](https://uqpyproject.readthedocs.io/en/latest/surrogates/pce/polynomial_bases.html). A hyperbolic truncation scheme is included below.

In [ ]:
# define and fit Polynomial Chaos Expansion
polynomial_basis = uq.TotalDegreeBasis(distribution, max_degree=2)
# polynomial_basis = uq.HyperbolicBasis(distribution, max_degree=4, hyperbolic=0.2)
regression_method = uq.LeastSquareRegression()
pce = uq.PolynomialChaosExpansion(polynomial_basis, regression_method)
pce.fit(x_train, y_train)

## 3. Compare statistics and Sobol sensitivities

Once trained, PCEs benefit from their polynomial form. We can quickly compute the mean and variance of PCE predictions using its polynomial coefficients. This provides a simple check for the behavior of our surrogate. The Monte Carlo estimates of mean and variance were computed by `compute_mc_statistics.py` using 10,000 samples draw from a Latin hypercube sampling scheme.

 The Sobol sensitivities, a measure of the "importance" of inputs to a model, can also be [estimated from the polynomial coefficients](https://uqpyproject.readthedocs.io/en/latest/sensitivity/pce.html). Using a PCE model, we can compute the Sobol sensitives in a fraction of a second, while the same computation using Monte Carlo estimates takes several minutes. The [Monte Carlo estimates of Sobol sensitivities](https://uqpyproject.readthedocs.io/en/latest/sensitivity/sobol.html) were computed by `compute_mc_statistics.py` using 2,000 samples per input dimension.

In [ ]:
pce_prediction = pce.predict(x_test)

pce_mean, pce_variance = pce.get_moments()
print("PCE Statistics", "Lift".ljust(10), "Torque")
print("Mean".ljust(14), f"{pce_mean[0]:5.4f} {pce_mean[3]:11.4f}")
print("Variance".ljust(14), f"{pce_variance[0]:.4e} {pce_variance[3]:.4e}")
print()

pce_sensitivity = uq.PceSensitivity(pce)
pce_sensitivity.run()

input_names = [
    "Angle of Attack",
    "Reynolds Number",
    "Upper Surface Trip Location",
    "Lower Surface Trip Location",
    "Flap Deflection"
]
print("PCE Sobol Indices".ljust(27), "Lift".ljust(6), "Torque")
for i in range(len(input_names)):
    sobol_lift = pce_sensitivity.first_order_indices[i, 0]
    sobol_torque = pce_sensitivity.first_order_indices[i, 3]
    print(input_names[i].ljust(27), f"{sobol_lift: .2f} {sobol_torque: 6.2f}")

```
MC Statistics Lift       Torque
Mean          0.2100     -0.0473
Variance      1.6483e-04 8.1500e-07
```

## 4. Plot results
 The hard work is done! We computed the training data, define and trained a PCE surrogate, then used the polynomial coefficients to estimate statistics on our outputs. The rest of this notebook compares the PCE predictions to the values from XFoil.

In [ ]:
import matplotlib.pyplot as plt

plt.style.use(["ggplot", "surg.mplstyle"])

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(7, 5))
ax0.scatter(
    x_test[:, 0],
    y_test[:, 0],
    label="XFoil Lift",
    color="darkgray",
    marker="o",
    s=10 ** 2,
)
ax0.scatter(
    x_test[:, 0],
    pce_prediction[:, 0],
    label="PCE Lift",
    color="tab:blue",
    marker="D",
    s=6 ** 2,
)
ax0.set_title("PCE Predictions for Lift")
ax0.set(xlabel="Angle of Attack", ylabel="Lift (CL)")
ax0.legend(loc="upper left", framealpha=1.0)
ax1.scatter(
    x_test[:, 4],
    y_test[:, 3],
    label="XFoil Torque",
    color="darkgray",
    marker="o",
    s=10 ** 2,
)
ax1.scatter(
    x_test[:, 4],
    pce_prediction[:, 3],
    label="PCE Torque",
    color="tab:blue",
    marker="D",
    s=6**2,
)
ax1.set_title("PCE Predictions for Torque")
ax1.set(xlabel="Flap Deflection", ylabel="Torque (CM)")
ax1.legend(loc="upper left", framealpha=1.0)
fig.suptitle("Polynomial Chaos Expansion Surrogate")
fig.tight_layout()
plt.show()

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(7, 4))
ax0.scatter(y_test[:, 0], pce_prediction[:, 0], color="tab:blue", alpha=0.4, zorder=2)
ax1.scatter(y_test[:, 3], pce_prediction[:, 3], color="tab:blue", alpha=0.4, zorder=2)

ax0.set_title("Comparison of Lift")
ax0.set(xlabel="XFoil Lift", ylabel="PCE Lift")
ax1.set_title("Comparison of Torque")
ax1.set(xlabel="XFoil Torque", ylabel="PCE Torque")
ax1.set_yticks(ax1.get_xticks())
for ax in (ax0, ax1):
    xlim = ax.get_xlim()
    ax.plot(xlim, xlim, color="white", linewidth=0.5, zorder=1)
    ax.set_aspect("equal")
    ax.set(xlim=xlim, ylim=xlim)
fig.suptitle("Polynomial Chaos Expansion Surrogate")
fig.tight_layout()

plt.show()